# Lesson 1: Retrieving All Records

Welcome to the lesson on retrieving all records using Django ORM! In previous lessons, we've covered the basics of CRUD (Create, Read, Update, Delete) operations. Now, we will delve deeper into the **Read** aspect, focusing on fetching data from your database. Retrieving data efficiently can significantly enhance the performance and usability of your application.

## What You'll Learn

In this lesson, we will explore how to retrieve all the records from your database using Django ORM. Understanding how to fetch all records is essential for scenarios where you need to display a list of tasks, users, or any other data stored in your database.

We'll use a simple **Todo** model for our examples. Here’s a quick look at the model:

```python
from django.db import models

class Todo(models.Model):
    task = models.CharField(max_length=200)
    completed = models.BooleanField(default=False)

    def __str__(self):
        return self.task
```

To retrieve all the records from the `Todo` model, we use the `all()` method provided by Django ORM. Here’s how we achieve that in a view function:

```python
from django.http import JsonResponse
from .models import Todo

def get_todos(request):
    todos = Todo.objects.all().values()
    return JsonResponse(list(todos), safe=False)
```

This function retrieves all `Todo` objects and returns them as a JSON response, making it easy to integrate with front-end applications.

## Why It Matters

Why is learning to retrieve all records important? In many applications, you will often need to display a list of items to the user. For example:
- An e-commerce site may need to show all products.
- A task management app may need to list all tasks.

Being adept at fetching data efficiently ensures your app performs well and provides a smooth user experience.

With a solid grasp of data retrieval, you can build dynamic features that respond to user interactions, such as displaying a user's activities, orders, or messages. Understanding this concept is a foundational skill that you will use in almost every Django project you work on.

---

Let's proceed to the practice section and apply what we've learned by retrieving all **Todo** records from our database. Exciting times are ahead! 🎉



## Retrieve All Todo Records

## Retrieve Only Incomplete Todos

To better understand how to retrieve records using Django ORM, this example breaks down the essential components:

### 1. **Models**
The `Todo` model defines a simple structure for storing tasks, with each task having a `task` (text field) and `completed` (boolean).

```python
from django.db import models

class Todo(models.Model):
    task = models.CharField(max_length=200)
    completed = models.BooleanField(default=False)

    def __str__(self):
        return self.task
```

### 2. **Views**
The views define functions for adding tasks and retrieving data from the database:

- `add_todo`: This function processes `POST` requests and adds a new task into the `Todo` table.
- `get_todos`: Retrieves all tasks from the `Todo` table.
- `get_completed`: Retrieves only the tasks marked as completed (`completed=True`).

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Todo
import json

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_todo = Todo(task=data['task'], completed=data.get('completed', False))
        new_todo.save()
        return JsonResponse({'id': new_todo.id, 'task': new_todo.task}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)

def get_todos(request):
    todos = Todo.objects.all().values()
    return JsonResponse(list(todos), safe=False)

def get_completed(request):
    todos = Todo.objects.filter(completed=True).values()
    return JsonResponse(list(todos), safe=False)
```

### 3. **URLs**
The URLs provide routes that map to the corresponding view functions. These URLs will be used by the frontend or external scripts to interact with the API.

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add_todo/', views.add_todo, name='add_todo'),
    path('todos/', views.get_todos, name='get_todos'),
    path('completed/', views.get_completed, name='get_completed'),
]
```

### 4. **send_request.py**
This script sends `POST` requests to add new `Todo` items and then retrieves all tasks and completed tasks using `GET` requests. It checks if tasks were already added using a flag file to prevent adding the same tasks again.

```python
import requests
import os

URL = 'http://127.0.0.1:3000'
FLAG_FILE = '.tasks_added.flag'

# Check if tasks have already been added
if not os.path.exists(FLAG_FILE):
    for todo in [('Buy groceries', False), ('Pay bills', False), ('Clean house', True)]:
        response = requests.post(URL + '/add_todo/', json={'task': todo[0], 'completed': todo[1]})
    # Create the flag file to indicate that tasks have been added
    with open(FLAG_FILE, 'w') as file:
        file.write('Tasks have been added')

response = requests.get(URL + '/todos/')
print('All todos: ', response.json())

response = requests.get(URL + '/completed/')
print('Completed todos: ', response.json())
```

### Steps to Run:
1. **Model**: The `Todo` model defines how the task data is stored in the database.
2. **Views**: Provides functionality to add, retrieve, and filter tasks based on the `completed` attribute.
3. **URLs**: Specifies the endpoints for adding and retrieving tasks.
4. **send_request.py**: Simulates interaction with the API by sending requests to add and fetch tasks.

### Conclusion:
This setup demonstrates how Django's ORM can be used to manage tasks and interact with a database. By running the provided code, you'll observe how tasks are added and how you can fetch them from the database using different endpoints.

## Running your Django ORM Code

To modify the `get_todos` view function so that it retrieves only the incomplete todos, you'll need to apply a filter in Django ORM based on the `completed` field being `False`. This can be done using the `filter()` method.

Here's the modified code for the `get_todos` view function:

### Modified `get_todos` Function

```python
def get_todos(request):
    # Retrieve only the incomplete todos (where completed=False)
    todos = Todo.objects.filter(completed=False).values()
    return JsonResponse(list(todos), safe=False)
```

### Explanation:
- **`filter(completed=False)`**: This line filters the `Todo` objects and retrieves only those where the `completed` field is `False` (i.e., incomplete todos).
- **`.values()`**: This method retrieves the dictionary representation of the records, which is returned as a JSON response.

### Full Code in `views.py`

```python
import json
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Todo

@csrf_exempt
def add_todos(request):
    if request.method == 'POST':
        try:
            data = json.loads(request.body)
        except json.JSONDecodeError:
            return JsonResponse({"message": "Invalid JSON."}, status=400)
        
        todos = data.get('todos', [])

        if todos:
            for todo in todos:
                title = todo.get('title', '')
                description = todo.get('description', '')
                completed = todo.get('completed', False)
                
                if not title or not description:
                    return JsonResponse({"message": "Title and description are required."}, status=400)
                
                new_todo = Todo(
                    title=title,
                    description=description,
                    completed=completed
                )
                new_todo.save()
            
            return JsonResponse({"message": "Todos created successfully!"}, status=201)
        else:
            return JsonResponse({"message": "No todos provided."}, status=400)
    
    return JsonResponse({"message": "Only POST method is allowed."}, status=405)

def get_todos(request):
    # Retrieve only the incomplete todos (where completed=False)
    todos = Todo.objects.filter(completed=False).values()
    return JsonResponse(list(todos), safe=False)
```

### Testing with `send_request.py`

After making this modification, you can use the `send_request.py` script to test whether only incomplete todos are being retrieved:

```python
import requests
import os

todos = [
    {
        "title": "Learn Django",
        "description": "Learn how to build web applications using Django.",
        "completed": False
    },
    {
        "title": "Build a Todo App",
        "description": "Build a simple Todo application using Django.",
        "completed": True
    },
    {
        "title": "Deploy Django App",
        "description": "Deploy the Todo application to a server.",
        "completed": False
    }
]

FLAG_FILE = '.todos_added.flag'

# Check if todos have already been added
if not os.path.exists(FLAG_FILE):
    # Sending the POST request with JSON data
    response = requests.post('http://localhost:3000/add-todos/', json={"todos": todos})
    if response.status_code == 201:
        # Create the flag file to indicate that todos have been added
        with open(FLAG_FILE, 'w') as file:
            file.write('Todos have been added')
    else:
        print("Failed to add todos.", response.text)

# Sending the GET request to retrieve only incomplete todos
response = requests.get('http://localhost:3000/get-todos/')
if response.status_code == 200:
    print("Incomplete todos retrieved successfully!")
    for todo in response.json():
        print(todo)
else:
    print("Failed to retrieve todos.")
```

### Expected Output

After running the script, it should only display the incomplete tasks (where `completed=False`):

```bash
Incomplete todos retrieved successfully!
{'title': 'Learn Django', 'description': 'Learn how to build web applications using Django.', 'completed': False}
{'title': 'Deploy Django App', 'description': 'Deploy the Todo application to a server.', 'completed': False}
```

### Conclusion
By modifying the `get_todos` function, you've successfully implemented filtering in Django ORM to retrieve specific records. This approach can be adapted for various filtering requirements depending on your application's needs.

## Modify Todo Retrieval Function

You've introduced a great way to apply more complex queries using Django's `Q` objects! Here's a detailed explanation of the key aspects and modifications you can use for filtering todos based on multiple conditions.

### Updated Code Explanation:

1. **Using Q Objects for Advanced Filtering**:
   - You can combine multiple conditions in a query using `Q` objects, which allows for more sophisticated queries such as OR conditions.
   - In this case, we're filtering todos that either:
     - Are completed (`completed=True`)
     - Or contain the word "Clean" in their task (`task__icontains="Clean"`, where `__icontains` is a case-insensitive search).

```python
from django.http import JsonResponse
from django.db.models import Q
from .models import Todo

def get_filtered_todos(request):
    # Get todos that are completed or contain the word "Clean" in the task
    todos = Todo.objects.filter(Q(completed=True) | Q(task__icontains="Clean")).values()
    return JsonResponse(list(todos), safe=False)
```

### Full Code Overview:

#### 1. **Views (`views.py`)**:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.db.models import Q
from .models import Todo
import json

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        if Todo.objects.filter(task=data['task']).exists():
            return JsonResponse({'message': 'Task already exists'}, status=400)
        new_todo = Todo(task=data['task'], completed=data.get('completed', False))
        new_todo.save()
        return JsonResponse({'id': new_todo.id, 'task': new_todo.task}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)

def get_todos(request):
    # Get all todos
    todos = Todo.objects.all().values()
    return JsonResponse(list(todos), safe=False)

def get_completed(request):
    # Get todos that are completed (completed=True)
    todos = Todo.objects.filter(completed=True).values()
    return JsonResponse(list(todos), safe=False)

def get_filtered_todos(request):
    # Get todos that are completed or contain the word "Clean" in the task
    todos = Todo.objects.filter(Q(completed=True) | Q(task__icontains="Clean")).values()
    return JsonResponse(list(todos), safe=False)
```

#### 2. **Model (`models.py`)**:

```python
from django.db import models

class Todo(models.Model):
    task = models.CharField(max_length=200)
    completed = models.BooleanField(default=False)

    def __str__(self):
        return self.task
```

#### 3. **Test Script (`send_request.py`)**:

```python
import requests
import os

URL = 'http://127.0.0.1:3000'
FLAG_FILE = '.todos_added.flag'

# Check if todos have already been added
if not os.path.exists(FLAG_FILE):
    for todo in [('Buy groceries', False), ('Pay bills', True), ('Clean house', True), ('Clean the balcony', False)]:
        response = requests.post(URL + '/add_todo/', json={'task': todo[0], 'completed': todo[1]})
    # Create the flag file to indicate that todos have been added
    with open(FLAG_FILE, 'w') as file:
        file.write('Todos have been added')

# Send requests to get todos, completed todos, and filtered todos
response = requests.get(URL + '/todos/')
print('All todos:', response.json())

response = requests.get(URL + '/completed/')
print('Completed todos:', response.json())

response = requests.get(URL + '/filtered/')
print('Filtered todos:', response.json())
```

#### 4. **URLs (`urls.py`)**:

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add_todo/', views.add_todo, name='add_todo'),
    path('todos/', views.get_todos, name='get_todos'),
    path('completed/', views.get_completed, name='get_completed'),
    path('filtered/', views.get_filtered_todos, name='get_filtered_todos'),
]
```

### Testing with `send_request.py`:

When you run the `send_request.py` script, it will:

1. **Add Todos**:
   - Add four tasks: "Buy groceries", "Pay bills", "Clean house", and "Clean the balcony".
   
2. **Get Todos**:
   - Retrieve all tasks using the `/todos/` endpoint.
   
3. **Get Completed Todos**:
   - Retrieve only completed tasks using the `/completed/` endpoint.

4. **Get Filtered Todos**:
   - Retrieve tasks that are either completed or contain the word "Clean" using the `/filtered/` endpoint.

### Expected Output:

The `/filtered/` endpoint should return tasks that meet the following criteria:
- Tasks marked as `completed=True`, such as "Pay bills" and "Clean house".
- Tasks containing the word "Clean", such as "Clean house" and "Clean the balcony".

Sample output for filtered todos:

```bash
Filtered todos: [
    {'task': 'Pay bills', 'completed': True},
    {'task': 'Clean house', 'completed': True},
    {'task': 'Clean the balcony', 'completed': False}
]
```

### Conclusion:
By using `Q` objects and the `__icontains` lookup, you've enhanced the query to allow for more complex and flexible filtering. This approach is useful for building more advanced search features or complex filtering logic in your Django application.

## Fix the Task Retrieval Bug

To modify the `get_completed` function so that it filters tasks that either contain the word "Clean" or "Wash" **and** are completed, we can use Django's `Q` objects. This will allow us to build a query that satisfies both conditions: one for task matching and one for the completion status.

Here's how you can modify the function:

### Updated `get_completed` function:

```python
from django.http import JsonResponse
from django.db.models import Q
from .models import Todo

def get_completed(request):
    # Filter for tasks that contain the word "Clean" or "Wash" AND are completed
    todos = Todo.objects.filter((Q(task__icontains="Clean") | Q(task__icontains="Wash")) & Q(completed=True)).values()
    return JsonResponse(list(todos), safe=False)
```

### Explanation:

1. **Conditions**:
   - `Q(task__icontains="Clean") | Q(task__icontains="Wash")`: This part checks whether the task contains either the word "Clean" or "Wash" (case-insensitive).
   - `& Q(completed=True)`: This ensures that the task is also marked as completed.
   
2. **Filter Logic**:
   - The `|` operator combines the two `Q` conditions (for "Clean" and "Wash"), meaning either condition can be true.
   - The `&` operator ensures that the task also satisfies the `completed=True` condition.

### Full Updated Code:

#### 1. **Views (`views.py`)**:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.db.models import Q
from .models import Todo
import json

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        if Todo.objects.filter(task=data['task']).exists():
            return JsonResponse({'message': 'Task already exists'}, status=400)
        new_todo = Todo(task=data['task'], completed=data.get('completed', False))
        new_todo.save()
        return JsonResponse({'id': new_todo.id, 'task': new_todo.task}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)

def get_todos(request):
    todos = Todo.objects.all().values()
    return JsonResponse(list(todos), safe=False)

def get_completed(request):
    # Filter for tasks that contain the word "Clean" or "Wash" AND are completed
    todos = Todo.objects.filter((Q(task__icontains="Clean") | Q(task__icontains="Wash")) & Q(completed=True)).values()
    return JsonResponse(list(todos), safe=False)
```

#### 2. **Model (`models.py`)**:

No changes are needed to the model, but here's a reference to the model in case you need it:

```python
from django.db import models

class Todo(models.Model):
    task = models.CharField(max_length=200)
    completed = models.BooleanField(default=False)

    def __str__(self):
        return self.task
```

#### 3. **Test Script (`send_request.py`)**:

Here’s how to test the updated function using a script:

```python
import requests
import os

URL = 'http://127.0.0.1:3000'
FLAG_FILE = '.todos_added.flag'

# Check if todos have already been added
if not os.path.exists(FLAG_FILE):
    todos = [
        ('Buy groceries', False),
        ('Pay bills', True),
        ('Clean house', True),
        ('Wash car', False),
        ('Wash clothes', True)
    ]
    
    for todo in todos:
        response = requests.post(URL + '/add_todo/', json={'task': todo[0], 'completed': todo[1]})
        print(f"Added: {todo[0]} - Status: {response.status_code}")
    
    # Create the flag file to indicate that todos have been added
    with open(FLAG_FILE, 'w') as file:
        file.write('Todos have been added')

# Get the completed todos that contain "Clean" or "Wash"
response = requests.get(URL + '/completed/')
print('Filtered Completed Todos:', response.json())
```

#### 4. **URLs (`urls.py`)**:

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add_todo/', views.add_todo, name='add_todo'),
    path('todos/', views.get_todos, name='get_todos'),
    path('completed/', views.get_completed, name='get_completed'),
]
```

### Conclusion:
With this update, the `get_completed` function will now return tasks that are completed and contain either the word "Clean" or "Wash". The `Q` object makes it easy to combine multiple conditions, allowing for flexible and complex filtering logic. 

The test script sends a few tasks to the server and retrieves only the completed ones that contain the words "Clean" or "Wash", ensuring the filtering works as expected.

## Retrieve Movie Data with Django

The issue with the code is in the `get_inactive_tasks` function, where the filtering logic is incorrect. Specifically, this line:

```python
tasks = Task.objects.filter(active=bool('False')).values()
```

The expression `bool('False')` will always evaluate to `True` because non-empty strings in Python are truthy. This means that all tasks are being treated as active, which is not the intended behavior.

### Fix:
To correctly filter for inactive tasks (where `active=False`), simply change the `filter()` clause to:

```python
tasks = Task.objects.filter(active=False).values()
```

This ensures that only tasks where `active` is `False` are retrieved.

### Final Code for `get_inactive_tasks`:
```python
from django.http import JsonResponse
from .models import Task
from django.db.models import Q

def get_inactive_tasks(request):
    tasks = Task.objects.filter(active=False).values()  # Correct filtering for inactive tasks
    return JsonResponse(list(tasks), safe=False)
```

### Explanation:
- The original code uses `bool('False')`, which always evaluates to `True` because any non-empty string is considered `True` in Python.
- The fix is to directly use `False` to filter tasks that are inactive (`active=False`), ensuring the correct results are returned.

### Conclusion:
By replacing `bool('False')` with `False` in the filter, the function will now properly return all tasks that are inactive.

To complete the Django views for retrieving all `Movie` objects and `Movie` objects with `oscar_won=True`, and update the `urls.py`, follow the steps below.

### Update the Views (`views.py`):

#### 1. `get_movies` View:
- This view should retrieve all `Movie` objects and return them in JSON format.

```python
def get_movies(request):
    movies = Movie.objects.all().values()  # Retrieve all movies
    return JsonResponse(list(movies), safe=False)
```

#### 2. `get_oscar_movies` View:
- This view should retrieve all `Movie` objects where `oscar_won=True` and return them in JSON format.

```python
def get_oscar_movies(request):
    oscar_movies = Movie.objects.filter(oscar_won=True).values()  # Retrieve movies that won an Oscar
    return JsonResponse(list(oscar_movies), safe=False)
```

### Updated `views.py`:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Movie
import json

@csrf_exempt
def add_movie(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_movie = Movie(title=data['title'], genre=data['genre'], oscar_won=data.get('oscar_won', False))
        new_movie.save()
        return JsonResponse({'id': new_movie.id, 'title': new_movie.title}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)

def get_movies(request):
    movies = Movie.objects.all().values()  # Retrieve all Movie objects
    return JsonResponse(list(movies), safe=False)

def get_oscar_movies(request):
    oscar_movies = Movie.objects.filter(oscar_won=True).values()  # Retrieve only Oscar-winning movies
    return JsonResponse(list(oscar_movies), safe=False)
```

### Update the URLs (`urls.py`):

#### 1. Define the endpoints for the views in `urls.py`:

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add_movie/', views.add_movie, name='add_movie'),
    path('get_movies/', views.get_movies, name='get_movies'),  # Endpoint for retrieving all movies
    path('get_oscar_movies/', views.get_oscar_movies, name='get_oscar_movies'),  # Endpoint for retrieving Oscar-winning movies
]
```

### Final URLs Setup (`urls.py`):

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add_movie/', views.add_movie, name='add_movie'),
    path('get_movies/', views.get_movies, name='get_movies'),
    path('get_oscar_movies/', views.get_oscar_movies, name='get_oscar_movies'),
]
```

### Conclusion:
- The `get_movies` view retrieves all movies from the database.
- The `get_oscar_movies` view filters and retrieves only the movies that have won an Oscar.
- The `urls.py` file defines the corresponding endpoints for these views.

This should now work correctly for listing all movies and filtering for Oscar-winning movies!